In [ ]:
install.packages("Rcpp")
install.packages("testthat")
install.packages("roxygen2")
install.packages("spatstat")

Updating HTML index of packages in '.Library'
Making 'packages.html' ... done
Updating HTML index of packages in '.Library'
Making 'packages.html' ... done
also installing the dependencies ‘ps’, ‘processx’, ‘callr’, ‘pkgbuild’, ‘pkgload’

Updating HTML index of packages in '.Library'
Making 'packages.html' ... done
also installing the dependency ‘polyclip’



In [ ]:
require(devtools)
require(Rcpp)
require(BH)
require(testthat)

In [ ]:
options(unzip = "internal")
devtools::install_github("YegorGalkin/RcppSim",quick=TRUE,local=FALSE)

In [ ]:
require(MathBioSim)

scales=10^(0:10/50)

areas=scales

pop<-list()

time_start<-Sys.time()

for(i in 1:length(scales)){
  
x_grid=(0:1000)/1000*0.002*scales[i]*10 
  
death_kernel=dnorm(x_grid,sd=0.001*scales[i])

birth_kernel=dnorm(x_grid,sd=0.002*scales[i])

params<-list("area_length_x"=areas[i],    
             "cell_count_x"=100,  
             
             "b"=0.3,    
             "d"=0,    
             "dd"=0.01*scales[i], 
             
             "seed"=1234,  
             "init_density"=100/scales[i],
             
             "death_kernel_x"=x_grid,
             "death_kernel_y"=death_kernel,
             
             "birth_kernel_x"=x_grid,
             "birth_kernel_y"=birth_kernel, 
             
             "spline_precision" = 1e-6  
)
sim<-new(poisson_1d,params)
pop[[i]]<-numeric(100000)
for(j in 1:(100000)){
  sim$make_event()
  pop[[i]][j]=sim$total_population
}
message(paste(i,"done"))
}

time_end<-Sys.time()
time_end-time_start

In [ ]:
require(ggplot2)
require(dplyr)

In [ ]:
mean_pop<-pop%>%sapply(function(x){x%>%tail(50000)%>%mean()})

In [ ]:
ggplot(data=data.frame(scales=scales,mean_pop=mean_pop),aes(x=scales,y=mean_pop))+geom_point()+scale_x_log10()

In [ ]:
#First simulation
ggplot(data=data.frame(event=1:100000,pop=pop[[1]]),aes(x=event,y=pop))+geom_line()

In [ ]:
#Fiftieth simulation
ggplot(data=data.frame(event=1:100000,pop=pop[[50]]),aes(x=event,y=pop))+geom_line()

In [ ]:
#Last simulation
ggplot(data=data.frame(event=1:100000,pop=pop[[151]]),aes(x=event,y=pop))+geom_line()

In [ ]:
plot_ith_density=function(i){
    death_kernel=dnorm(x_grid,sd=0.001*scales[i])
    birth_kernel=dnorm(x_grid,sd=0.002*scales[i])

params<-list("area_length_x"=areas[i],    
             "cell_count_x"=100,  
             
             "b"=0.3,    
             "d"=0,    
             "dd"=0.01*scales[i], 
             
             "seed"=1234,  
             "init_density"=100/scales[i],
             
             "death_kernel_x"=x_grid,
             "death_kernel_y"=death_kernel,
             
             "birth_kernel_x"=x_grid,
             "birth_kernel_y"=birth_kernel, 
             
             "spline_precision" = 1e-6  
)
sim<-new(poisson_1d,params)

all_coords<-list()

for(j in 1:(100000)){
  sim$make_event()
  if(j%%1000==0) all_coords[[j%/%1000]]<-sim$get_all_coordinates()
}
ggplot(data=data.frame(coords=unlist(all_coords%>%tail(50))),aes(coords))+geom_histogram(bins=60)
}

In [ ]:
plot_ith_density(1)

In [ ]:
plot_ith_density(50)

In [ ]:
plot_ith_density(150)

In [ ]:
scales=10^(0:100/50)

areas=scales

pop<-list()

time_start<-Sys.time()

for(i in 1:length(scales)){
  
x_grid=(0:1000)/1000*0.002*10 
  
death_kernel=dnorm(x_grid,sd=0.001)

birth_kernel=dnorm(x_grid,sd=0.002)

params<-list("area_length_x"=areas[i],    
             "cell_count_x"=100,  
             
             "b"=0.3,    
             "d"=0,    
             "dd"=0.01, 
             
             "seed"=1234,  
             "init_density"=100,
             
             "death_kernel_x"=x_grid,
             "death_kernel_y"=death_kernel,
             
             "birth_kernel_x"=x_grid,
             "birth_kernel_y"=birth_kernel, 
             
             "spline_precision" = 1e-6  
)
sim<-new(poisson_1d,params)
pop[[i]]<-numeric(100000*sqrt(scales[i]))
for(j in 1:(100000*sqrt(scales[i]))){
  sim$make_event()
  pop[[i]][j]=sim$total_population
}
message(paste(i,"done"))
}

time_end<-Sys.time()
time_end-time_start

In [ ]:
mean_pop<-pop%>%sapply(function(x){x%>%tail(50000)%>%mean()})

In [ ]:
ggplot(data=data.frame(scales=scales,mean_pop=mean_pop),aes(x=scales,y=mean_pop))+geom_point()

In [ ]:
lm(mean_pop~scales-1)

In [ ]:
install.packages("spatstat")

In [ ]:
require(spatstat)
points<-unique.ppp(ppp(sim$get_all_coordinates(),rep(0,sim$total_population),c(0,100),c(-50,50)))

K_estimate<-Kest(points,r=x_grid,correction="Ripley")

ggplot(data=data.frame(Kest=K_estimate$iso/2,x=x_grid),aes(x=x,y=Kest))+geom_line()

In [ ]:
pcf_estimate=data.frame(Kest=K_estimate$iso/2,x=x_grid)%>%
  mutate(pfc=(Kest-lag(Kest))/(x_grid-lag(x_grid))/sim$area_length_x)

ggplot(data=pcf_estimate,aes(x=x,y=pfc))+geom_line()+
  labs(x="r",y="PCF")